In [1]:
import numpy as np
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))
from utils import de, dcde
from benchmark.simple import rosenbrock, rastrigin, shubert
from IPython.display import clear_output
import time

In [2]:
runs = 1000

populationSize = 20
boundaries = [(-1000,1000)]
dimensions = 200
mutationFactor = 0.5
crossingRate = 0.5
fitness = rastrigin
generations = 1500

In [3]:
dcde2Fitness = np.zeros(runs)
dcdeFitness = np.zeros(runs)
deFitness = np.zeros(runs)

dcde2Time = np.zeros(runs)
dcdeTime = np.zeros(runs)
deTime = np.zeros(runs)

globalStart = time.time()
run = 0
for r in range(runs):

    # regular DE
    deStart = time.time()
    deBest = de.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        generations = 2*generations # double since divide and conquer has extras generations on the divide cycle
    )
    deEllapsed = time.time() - deStart

    # divide and conquer DE
    dcdeStart = time.time()
    dcdeBest = dcde.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        generations = generations,
    )
    dcdeEllapsed = time.time() - dcdeStart
    
    # divide and conquer DE + increase mutation on conquer step
    dcde2Start = time.time()
    dcde2Best = dcde.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        generations = generations,
        increaseMutation = True,
    )
    dcde2Ellapsed = time.time() - dcde2Start
    
    dcde2Fitness[r] = fitness(dcde2Best)
    dcdeFitness[r] = fitness(dcdeBest)
    deFitness[r]   = fitness(deBest)

    dcde2Time[r] = dcde2Ellapsed    
    dcdeTime[r] = dcdeEllapsed
    deTime[r]   = deEllapsed
    
    run = r + 1  
    deMean = np.mean(deFitness[:run])
    deStd = np.std(deFitness[:run])
    deMin = np.min(deFitness[:run])
    deMax = np.max(deFitness[:run])
    deMeanTime = np.mean(deTime[:run])
    
    dcdeMean = np.mean(dcdeFitness[:run])
    dcdeStd = np.std(dcdeFitness[:run])
    dcdeMin = np.min(dcdeFitness[:run])
    dcdeMax = np.max(dcdeFitness[:run])
    dcdeMeanTime = np.mean(dcdeTime[:run])
  
    dcde2Mean = np.mean(dcde2Fitness[:run])
    dcde2Std = np.std(dcde2Fitness[:run])
    dcde2Min = np.min(dcde2Fitness[:run])
    dcde2Max = np.max(dcde2Fitness[:run])
    dcde2MeanTime = np.mean(dcde2Time[:run])

    ellapsedTime = time.time() - globalStart
    remaining = (ellapsedTime/run) * (runs - run)
    ellapsedTimeStr = time.strftime('%H:%M:%S', time.gmtime(ellapsedTime))
    remainingStr = time.strftime('%H:%M:%S', time.gmtime(remaining))
    
    clear_output()
    print(f'Run {run}, Ellapsed: {ellapsedTimeStr}, Remaining: {remainingStr}')
    print(f'DE:     Mean = {deMean:.4f}, Std = {deStd:.4f}, Min = {deMin:.4f}, Max = {deMax:.4f}, Time = {deMeanTime:.4f}s')
    print(f'DCDE:   Mean = {dcdeMean:.4f}, Std = {dcdeStd:.4f}, Min = {dcdeMin:.4f}, Max = {dcdeMax:.4f}, Time = {dcdeMeanTime:.4f}s')
    print(f'DCDE2:  Mean = {dcde2Mean:.4f}, Std = {dcde2Std:.4f}, Min = {dcde2Min:.4f}, Max = {dcde2Max:.4f}, Time = {dcde2MeanTime:.4f}s')


Run 1000, Ellapsed: 02:04:58, Remaining: 00:00:00
DE:     Mean = 15682.8766, Std = 10363.8742, Min = 6211.8844, Max = 156945.6401, Time = 3.6148s
DCDE:   Mean = 5637.4341, Std = 333.3187, Min = 4476.5300, Max = 6644.7275, Time = 1.9355s
DCDE2:  Mean = 5654.7647, Std = 332.4400, Min = 4668.7260, Max = 6726.2897, Time = 1.9469s
